In [1]:
import pandas as pd
import numpy as np

# Load Excel file
xls = pd.ExcelFile("Hypothesis 2 data.xlsx")

# --- Step 1: Load raw date ranges ---
rppi_raw = xls.parse('RPPI')[['Month', 'VALUE']].rename(columns={'Month': 'Date', 'VALUE': 'RPPI'})
rppi_raw['Date'] = pd.to_datetime(rppi_raw['Date'])
rppi_min, rppi_max = rppi_raw['Date'].min(), rppi_raw['Date'].max()

gdp_raw = xls.parse('GDP').rename(columns={'observation_date': 'Date'})
gdp_raw['Date'] = pd.to_datetime(gdp_raw['Date'])
gdp_min, gdp_max = gdp_raw['Date'].min(), gdp_raw['Date'].max()

cpi_raw = xls.parse('CPI')[['Month', 'VALUE']].rename(columns={'Month': 'Date', 'VALUE': 'CPI'})
cpi_raw['Date'] = pd.to_datetime(cpi_raw['Date'])
cpi_min, cpi_max = cpi_raw['Date'].min(), cpi_raw['Date'].max()

interest_raw = xls.parse('Interest Rates').rename(columns={'Reporting date': 'Date', 'rates': 'InterestRate'})
interest_raw['Date'] = pd.to_datetime(interest_raw['Date'], dayfirst=True)
interest_min, interest_max = interest_raw['Date'].min(), interest_raw['Date'].max()

income_raw = xls.parse('disposable income')[['Year', 'VALUE']].rename(columns={'VALUE': 'DisposableIncome'})
income_raw['Date'] = pd.to_datetime(income_raw['Year'].astype(str) + '-01-01')
income_min, income_max = income_raw['Date'].min(), income_raw['Date'].max()

mortgage_raw = xls.parse('Morgtage approvals')
mortgage_raw.columns = ['Year', 'MonthName', 'MortgageApprovals']
months = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
          'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
mortgage_raw['Month'] = mortgage_raw['MonthName'].map(months)
mortgage_raw['Date'] = pd.to_datetime(mortgage_raw[['Year', 'Month']].assign(DAY=1))
mortgage_min, mortgage_max = mortgage_raw['Date'].min(), mortgage_raw['Date'].max()

# ---- ✅ Population (NEW) ----
population_raw = xls.parse('Population')
population_raw = population_raw[population_raw['County'] == 'State']
population_raw = population_raw[population_raw['Sex'] == 'Both sexes']
population_raw = population_raw[['CensusYear', 'VALUE']].rename(columns={'CensusYear': 'Year', 'VALUE': 'Population'})
population_raw['Date'] = pd.to_datetime(population_raw['Year'].astype(str) + '-01-01')
population = population_raw[['Date', 'Population']].set_index('Date').resample('MS').ffill().bfill().reset_index()
pop_min, pop_max = population['Date'].min(), population['Date'].max()

# --- Step 2: Final common date range ---
min_common = max([rppi_min, gdp_min, cpi_min, interest_min, income_min, mortgage_min, pop_min])
max_common = min([rppi_max, gdp_max, cpi_max, interest_max, income_max, mortgage_max, pop_max])
print(f"✅ TRUE Common Date Range: {min_common.strftime('%Y-%m')} to {max_common.strftime('%Y-%m')}")

# --- Step 3: Filter/clean each dataset to date range ---
rppi = rppi_raw[rppi_raw['Date'].between(min_common, max_common)]
gdp = gdp_raw[gdp_raw['Date'].between(min_common, max_common)].set_index('Date').resample('MS').ffill().reset_index()
cpi = cpi_raw[cpi_raw['Date'].between(min_common, max_common)]
interest = interest_raw[interest_raw['Date'].between(min_common, max_common)].set_index('Date').resample('MS').ffill().reset_index()
income = income_raw[income_raw['Date'].between(min_common, max_common)].set_index('Date').resample('MS').ffill().reset_index()
mortgage = mortgage_raw[mortgage_raw['Date'].between(min_common, max_common)][['Date', 'MortgageApprovals']]
population = population[population['Date'].between(min_common, max_common)]

# --- Step 4: Merge all ---
merged_df = rppi.merge(gdp, on='Date', how='left') \
                .merge(cpi, on='Date', how='left') \
                .merge(interest, on='Date', how='left') \
                .merge(income, on='Date', how='left') \
                .merge(mortgage, on='Date', how='left') \
                .merge(population, on='Date', how='left')

# Fill missing values
merged_df[['GDP', 'CPI', 'InterestRate', 'DisposableIncome', 'MortgageApprovals', 'Population']] = \
    merged_df[['GDP', 'CPI', 'InterestRate', 'DisposableIncome', 'MortgageApprovals', 'Population']].interpolate().ffill().bfill()

# --- Step 5: Policy Flags ---
merged_df['HTB_Active'] = (merged_df['Date'] >= pd.Timestamp('2017-01-01')).astype(int)
merged_df['HTB_Enhanced'] = (merged_df['Date'] >= pd.Timestamp('2020-07-01')).astype(int)
merged_df['RPZ_Active'] = (merged_df['Date'] >= pd.Timestamp('2016-12-01')).astype(int)
merged_df['RPZ_InflationLinked'] = (merged_df['Date'] >= pd.Timestamp('2021-07-01')).astype(int)

# --- Step 6: Save Output ---
merged_df.to_csv("final_dataset_true_overlap.csv", index=False)
print("📊 Final Shape:", merged_df.shape)
print("✅ Dataset with Population saved as 'final_dataset_true_overlap.csv'")


✅ TRUE Common Date Range: 2014-12 to 2022-01
📊 Final Shape: (85, 13)
✅ Dataset with Population saved as 'final_dataset_true_overlap.csv'


/var/folders/80/dpnwd7h93_s2rmdn2t1q5n0m0000gn/T/ipykernel_1111/2858024151.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  interest_raw['Date'] = pd.to_datetime(interest_raw['Date'], dayfirst=True)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Copy dataset
df = merged_df.copy()

# --- Step 1: Create Lag Features ---
df['RPPI_lag1'] = df['RPPI'].shift(1)
df['GDP_lag1'] = df['GDP'].shift(1)
df['CPI_lag1'] = df['CPI'].shift(1)
df['InterestRate_lag1'] = df['InterestRate'].shift(1)
df['DisposableIncome_lag1'] = df['DisposableIncome'].shift(1)
df['MortgageApprovals_lag1'] = df['MortgageApprovals'].shift(1)

# --- Step 2: Drop NA caused by shifting ---
df = df.dropna()
print("✅ Shape after lag creation:", df.shape)

# --- Step 3: Define Features & Target ---
X = df[['GDP_lag1', 'CPI_lag1', 'InterestRate_lag1', 'DisposableIncome_lag1', 'MortgageApprovals_lag1',
        'HTB_Active', 'HTB_Enhanced', 'RPZ_Active', 'RPZ_InflationLinked', 'RPPI_lag1']]
y = df['RPPI']

# --- Step 4: Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# --- Step 5: Initialize Models ---
models = {
    "OLS": LinearRegression(),
    "SVR": SVR(kernel='rbf'),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

# --- Step 6: Train and Evaluate ---
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results.append({
        "Model": name,
        "RMSE": round(rmse, 2),
        "MAE": round(mae, 2),
        "R² Score": round(r2, 4)
    })

# --- Step 7: Display Results ---
results_df = pd.DataFrame(results)
print("📈 Model Performance with Lag Features:")
print(results_df)


✅ Shape after lag creation: (84, 19)
📈 Model Performance with Lag Features:
           Model   RMSE    MAE  R² Score
0            OLS   1.20   0.96    0.9766
1            SVR  17.39  15.73   -3.9337
2  Random Forest  11.74   9.06   -1.2509
3        XGBoost  12.18   9.64   -1.4225
